# 🦴🌿 Chatbot "Tribu Salud" – Salud Evolutiva con IA 🏋️‍♂️🔥

## 🎯 Objetivo  
Desarrollar un **chatbot especializado en salud evolutiva** llamado **Tribu Salud**, utilizando **Mistral-7B-Instruct-v0.2**, con capacidades de comprensión y generación de respuestas basadas en principios ancestrales de alimentación, entrenamiento y bienestar.

---

## 🛠️ Tecnologías y Metodología  

### 🔹 1. Modelo de Lenguaje  
Se utiliza el modelo **Mistral-7B-Instruct-v0.2**, optimizado para respuestas conversacionales en español y otras lenguas.

### 🔹 2. Entrenamiento y Fine-Tuning  
- Se usa el dataset **MLQA** (*MultiLingual Question Answering*) para entrenamiento inicial.  
- Se aplica **fine-tuning** con datos específicos de salud evolutiva.  

### 🔹 3. RAG (Retrieval-Augmented Generation)  
Para mejorar la precisión de las respuestas, se implementa **RAG**, combinando el modelo generativo con una base de conocimiento extraída de:  
- **Documentos PDF** relevantes.  
- **Transcripciones de YouTube** de **Fitness Revolucionario**, obtenidas con `youtube-transcript-api`.  

### 🔹 4. Indexado con FAISS  
- Se mejora el **índice FAISS** para permitir **búsquedas rápidas y eficientes** en la base de conocimiento.  
- Esto permite respuestas basadas en información real y verificada.  

### 🔹 5. Despliegue con Gradio  
- Se utiliza **Gradio** para desplegar el chatbot con una interfaz accesible y fácil de usar.  

---

## 🚀 Estado y Próximos Pasos  
✅ Modelo Mixtral-8x22B cargado y configurado.  
✅ Dataset MLQA procesado y adaptado.  
✅ Implementación de Fine-Tuning.  
✅ Extracción de información de PDFs y YouTube.  
✅ Optimización de FAISS para búsqueda rápida.  
🔜 Despliegue final y pruebas en Gradio.  

---

## 📌 Conclusión  
**Tribu Salud** será un chatbot avanzado que proporcionará información precisa y confiable sobre **salud evolutiva, alimentación y entrenamiento funcional**, integrando un **modelo de IA con recuperación de información relevante**.  

📢 **¡Pronto estará disponible para consulta y uso!** 🚀🔥  


In [11]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 69.1 MB/s eta 0:00:00


In [12]:
!pip install -U langchain-community

In [13]:
!pip install youtube_transcript_api

In [14]:
!pip install datasets

In [15]:
!pip install gradio

In [16]:
!pip uninstall -y bitsandbytes
!pip install --no-cache-dir -U bitsandbytes

Found existing installation: bitsandbytes 0.45.1
Uninstalling bitsandbytes-0.45.1:
  Successfully uninstalled bitsandbytes-0.45.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 333.3 MB/s eta 0:00:00


In [17]:
!pip install -U peft

In [18]:
!pip install PyPDF2

In [19]:
!pip install pdfplumber

In [20]:
import bitsandbytes as bnb
print(bnb.__version__)

0.45.1


In [ ]:
import torch
print(torch.cuda.is_available())  # Debe devolver True
print(torch.cuda.get_device_name())  # Debe listar la A100

True
NVIDIA A100-SXM4-40GB


In [21]:
import torch
import os
import shutil
import logging
import json
import pdfplumber
import requests
import io
import faiss
import pickle
import zipfile
import numpy as np
import gradio as gr
from datasets import load_dataset
from transformers import (TrainingArguments, Trainer, AutoTokenizer, AutoModelForCausalLM,
                          DataCollatorForLanguageModeling, BitsAndBytesConfig, pipeline)
from huggingface_hub import HfApi, create_repo
from google.colab import files
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer

In [27]:
# Autenticación en Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Cargar modelo Mistral-7B-Instruct
# Configuración
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Cuantización en 4-bit
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,  # Usa bfloat16 en A100
    bnb_4bit_use_double_quant=True,
)

# Cargar modelo con offloading y LoRA
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

# Preparar modelo para fine-tuning en 4-bit
model = prepare_model_for_kbit_training(model)

# Configurar LoRA (añadir adaptadores en capas clave)
lora_config = LoraConfig(
    r=16,  # Tamaño del cuello de botella de LoRA (ajustar según RAM disponible)
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # Aplicar LoRA solo en capas clave
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"  # Tipo de tarea para el modelo
)

# Aplicar LoRA
model = get_peft_model(model, lora_config)

# Cargar tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Evitar errores con padding

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Mensaje de bienvenida del chatbot
system_message = """
🦴🌿 ¡Bienvenido a Tribu Salud! 🏋️‍♂️🔥

Hola, soy tu asistente en salud evolutiva, aquí para ayudarte a optimizar tu bienestar con un enfoque basado en la biología ancestral. 🏕️🥩

🔹 ¿Quieres mejorar tu alimentación con principios evolutivos? 🍖🥑
🔹 ¿Buscas consejos sobre entrenamiento funcional y movimiento natural? 🏃‍♂️💪
🔹 ¿Te interesa mejorar tu descanso y reducir el estrés? 😴🌞

Pregúntame lo que necesites, ¡empecemos este viaje hacia una salud más alineada con nuestra naturaleza! 🚀💯
"""

def generate_response(prompt):
    """Genera respuesta en español usando Mistral-7B-Instruct-v0.2."""
    full_prompt = f"<s>[INST] {system_message}\n{prompt}\nPor favor, responde en español. [/INST]"

    inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=256, do_sample=True, temperature=0.7, top_p=0.9)

    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # Eliminar el prompt de la respuesta final
    response = response.replace(full_prompt, "").strip()
    return response

# Prueba del chatbot
pregunta = "¿Qué beneficios tiene la dieta paleo?"
respuesta = generate_response(pregunta)
print(respuesta)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] 
🦴🌿 ¡Bienvenido a Tribu Salud! 🏋️‍♂️🔥

Hola, soy tu asistente en salud evolutiva, aquí para ayudarte a optimizar tu bienestar con un enfoque basado en la biología ancestral. 🏕️🥩

🔹 ¿Quieres mejorar tu alimentación con principios evolutivos? 🍖🥑
🔹 ¿Buscas consejos sobre entrenamiento funcional y movimiento natural? 🏃‍♂️💪
🔹 ¿Te interesa mejorar tu descanso y reducir el estrés? 😴🌞

Pregúntame lo que necesites, ¡empecemos este viaje hacia una salud más alineada con nuestra naturaleza! 🚀💯

¿Qué beneficios tiene la dieta paleo?
Por favor, responde en español. [/INST] La dieta paleo puede tener beneficios para la salud, como mejorar el equilibrio de macronutrientes, reducir la inflamación, mejorar la sensibilidad al glúcose y mejorar la insulina, y mejorar la calidad del sonido. Sin embargo, es importante mencionar que la dieta paleo no es una dieta balanceada por sí misma y puede requerir planificar cuidadosamente para garantizar que se obtienen todos los nutrientes necesarios. Además,

In [ ]:
# Cargar dataset
try:
    dataset = load_dataset("mlqa", "mlqa-translate-train.es", trust_remote_code=True)
    train_dataset = dataset["train"].shuffle(seed=42).select(range(min(10000, len(dataset["train"]))))
    val_dataset = (
        dataset["validation"].shuffle(seed=42).select(range(min(2000, len(dataset["validation"]))))
        if "validation" in dataset
        else None
    )
    print("✅ Dataset cargado correctamente.")
except Exception as e:
    print(f"❌ Error cargando dataset: {e}")
    train_dataset, val_dataset = None, None

# Función para generar prompt con formato Mistral
def create_prompt(sample):
    """
    Crea un prompt formateado para Mistral-7B-Instruct-v0.2 basado en un ejemplo del dataset.
    """
    prompt = (
        f"<s>[INST] {system_message}\n"
        f"Pregunta: {sample['question']}\n"
        f"Contexto: {sample['context']}\n"
        "Da una respuesta clara en español. [/INST]"
    )
    return prompt

# Función para tokenizar el prompt y preparar los labels.
def tokenize_prompt(sample):
    prompt = create_prompt(sample)
    tokenized = tokenizer(
        prompt,
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    tokenized["prompt"] = prompt  # Guardamos el prompt original
    return tokenized

# Aplicar la tokenización al dataset y eliminar columnas originales
if train_dataset is not None:
    train_dataset = train_dataset.map(
        tokenize_prompt,
        remove_columns=train_dataset.column_names,
        batched=False  # Procesamos ejemplo a ejemplo para mayor claridad
    )
if val_dataset is not None:
    val_dataset = val_dataset.map(
        tokenize_prompt,
        remove_columns=val_dataset.column_names,
        batched=False
    )

# Función para probar el modelo con una muestra del dataset tokenizado
def test_dataset_sample(dataset, index=0):
    """
    Prueba el modelo con una muestra del dataset MLQA ya tokenizado.
    """
    if dataset is None or len(dataset) <= index:
        print("❌ El dataset está vacío o el índice es inválido.")
        return

    sample = dataset[index]
    # Recuperamos el prompt original guardado durante la tokenización
    prompt = sample.get("prompt", "No se encontró prompt en la muestra.")

    # Configurar el dispositivo (CPU/GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Convertir los input_ids y attention_mask a tensores y agregar dimensión batch
    inputs = {
        "input_ids": torch.tensor(sample["input_ids"]).unsqueeze(0).to(device),
        "attention_mask": torch.tensor(sample["attention_mask"]).unsqueeze(0).to(device),
    }

    # Generar respuesta
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # Remover el prompt si aparece en la salida
    response = response.replace(prompt, "").strip()

    print("📌 Prompt generado:")
    print(prompt)
    print("\n🤖 Respuesta del modelo:")
    print(response)

# Probar con un ejemplo del dataset de entrenamiento (ya tokenizado)
test_dataset_sample(train_dataset, index=3)

✅ Dataset cargado correctamente.


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📌 Prompt generado:
<s>[INST] 
🦴🌿 ¡Bienvenido a Tribu Salud! 🏋️‍♂️🔥

Hola, soy tu asistente en salud evolutiva, aquí para ayudarte a optimizar tu bienestar con un enfoque basado en la biología ancestral. 🏕️🥩

🔹 ¿Quieres mejorar tu alimentación con principios evolutivos? 🍖🥑
🔹 ¿Buscas consejos sobre entrenamiento funcional y movimiento natural? 🏃‍♂️💪
🔹 ¿Te interesa mejorar tu descanso y reducir el estrés? 😴🌞

Pregúntame lo que necesites, ¡empecemos este viaje hacia una salud más alineada con nuestra naturaleza! 🚀💯

Pregunta: Por qué razón el primer ministro passos coelho justificó cortar 30000 puestos de trabajo?
Contexto: En la primera semana de mayo de 2013, el primer ministro passos coelho anunció un plan de gobierno significativo para el sector público, en el que se va 30,000 puestos de trabajo y el número de horas de trabajo semanales se aumentará de 35 a 40 horas. Coelho reafirmó el anuncio explicando que las medidas de austeridad son necesarias si Portugal busca evitar otra subvenc

In [ ]:
# Configuración de entrenamiento ajustada para reducir tiempo
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,  # Aumentar tamaño de batch para procesar más ejemplos por iteración
    per_device_eval_batch_size=8,  # Aumentamos también para la evaluación
    eval_strategy="no",  # Desactivar evaluación para evitar sobrecargar el tiempo de entrenamiento
    save_strategy="no",  # Desactivar guardar el modelo cada época para evitar tiempos adicionales
    num_train_epochs=1,  # Reducimos a 1 época para acelerar el entrenamiento
    learning_rate=5e-5,  # Aumentamos ligeramente la tasa de aprendizaje para entrenamiento más rápido
    weight_decay=0.01,
    bf16=True,  # A100 usa bfloat16 para mejorar la velocidad
    gradient_accumulation_steps=1,  # No es necesario acumular gradientes si se usa un batch más grande
    push_to_hub=False,
    gradient_checkpointing=False,  # Desactivamos checkpointing para acelerar
    optim="adamw_torch",  # Optimizador eficiente
    report_to="none",  # Desactiva logs en WandB
    remove_unused_columns=False,  # Dejamos False ya que ya eliminamos columnas en el map
)

# Crear Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=None,  # No necesario para modelos de lenguaje
)

# Iniciar entrenamiento
if train_dataset:
    print("🚀 Iniciando entrenamiento...")
    trainer.train()
    print("✅ Entrenamiento completado.")


🚀 Iniciando entrenamiento...


Step,Training Loss
500,0.933000
1000,0.880900


✅ Entrenamiento completado.


In [ ]:
# Guardar modelo entrenado localmente
model_dir = "/content/trained_tribu_salud"
os.makedirs(model_dir, exist_ok=True)

# Guardar el modelo y el tokenizador
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

# Comprimir la carpeta del modelo en un ZIP
zip_filename = "/content/trained_tribu_salud.zip"
shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', model_dir)
print(f"✅ Modelo guardado y comprimido en: {zip_filename}")

# Descargar el archivo ZIP localmente
files.download(zip_filename)
print("📥 Descarga iniciada...")

✅ Modelo guardado y comprimido en: /content/trained_tribu_salud.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Descarga iniciada...


In [ ]:
# Subir modelo a Hugging Face

repo_name = "CasiAC/chatbot_tribu_salud"
repo_dir = "/content/hf_tribu_salud"

# Verificar si el modelo existe antes de subirlo
if not os.path.exists(model_dir):
    raise FileNotFoundError(f"⚠ Error: No se encontró la carpeta {model_dir}. Verifica si el entrenamiento fue exitoso.")
else:
    print(f"✅ Carpeta del modelo encontrada: {model_dir}")

# Crear repositorio en Hugging Face (si no existe)
api = HfApi()
create_repo(repo_name, repo_type="model", private=True, exist_ok=True)

# Copiar modelo a la carpeta del repositorio
shutil.copytree(model_dir, repo_dir, dirs_exist_ok=True)

# Subir modelo a Hugging Face
api.upload_folder(
    folder_path=repo_dir,
    repo_id=repo_name,
    repo_type="model",
    commit_message="🚀 Subida inicial del chatbot Tribu Salud basado en Mistral 7B"
)

print(f"🚀 Modelo subido correctamente a Hugging Face: https://huggingface.co/{repo_name}")

✅ Carpeta del modelo encontrada: /content/trained_tribu_salud


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

🚀 Modelo subido correctamente a Hugging Face: https://huggingface.co/CasiAC/chatbot_tribu_salud


In [22]:
# Configuración de logging
logging.basicConfig(level=logging.INFO)

# 1️⃣ CARGA DE DOCUMENTOS (PDF y YouTube)
# ------------------------------------

pdf_url = "https://s3.amazonaws.com/fitnessrevolucionario.publico/Descargas/ElManualRevolucionario_MarcosVazquez.pdf"

# Cargar el PDF con pdfplumber
def load_pdf(pdf_url):
    try:
        # Descargar el PDF (o leer si ya lo tienes en el sistema)
        response = requests.get(pdf_url)
        pdf_file = io.BytesIO(response.content)  # Usamos BytesIO para leer desde memoria
        with pdfplumber.open(pdf_file) as pdf:
            pdf_text = [page.extract_text() for page in pdf.pages]
        return pdf_text
    except Exception as e:
        logging.error(f"Error al cargar el PDF: {e}")
        return []

pdf_pages = load_pdf(pdf_url)

# Lista de URLs de YouTube
youtube_urls = [
    "https://www.youtube.com/watch?v=L3Qf63iP4Yc",
    "https://www.youtube.com/watch?v=WT-wEIo9Ji8",
    "https://www.youtube.com/watch?v=gaVQ0lFp_Po",
    "https://www.youtube.com/watch?v=dDaKhCTrMus",
    "https://www.youtube.com/watch?v=DPOLIiBcCbk"
]

# Función para guardar las transcripciones en formato .srt
def save_to_srt(transcript, filename):
    with open(filename, 'w') as f:
        for idx, entry in enumerate(transcript, 1):
            start, duration, text = entry['start'], entry['duration'], entry['text']
            start_min, start_sec = divmod(int(start), 60)
            end_min, end_sec = divmod(int(start + duration), 60)
            f.write(f"{idx}\n")
            f.write(f"{start_min:02d}:{start_sec:02d} --> {end_min:02d}:{end_sec:02d}\n")
            f.write(f"{text}\n\n")

documents = [page for page in pdf_pages if page is not None]  # Eliminar páginas vacías

# Procesar cada URL de YouTube
for url in youtube_urls:
    video_id = url.split('v=')[1]
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['es'])
        transcript_filename = f"transcript_{video_id}.srt"
        save_to_srt(transcript, transcript_filename)
        documents.append(f"Transcripción de {url} guardada en {transcript_filename}")
    except (TranscriptsDisabled, NoTranscriptFound):
        logging.warning(f"No se encontró transcripción para {url}")
    except Exception as e:
        logging.error(f"Error al obtener transcripción para {url}: {e}")

# Imprimir los documentos procesados
for document in documents:
    print(document)

EL MANUALEl Manual Revolucionario 1
REVOLUCIONARIO
El principio Fitness
de tu revolución Revolucionario
Índice El Manual Revolucionario 2
Copyright © 2021 por
Fitness Revolucionario.
LAS BASES 3
NUTRICIÓN 6
17 ENTRENAMIENTO
OTROS ASPECTOS 20
21 RECETAS
Las Bases El Manual Revolucionario 3
LAS BASES
Una nueva forma de entender y
mejorar tu salud. Si este es tu primer
contacto con Fitness Revolucionario,
déjame resumirte lo que encontrarás
en mi blog y podcast.
01 02
Explicaciones detalladas sobre cómo Cuestionamientos de muchas creen-
funciona realmente tu cuerpo. Cui­ cias comunes sobre salud. Muchas
damos más aquello que conocemos, ideas que se repiten constantemente
y la mayoría desconoce los aspectos (como que hay que comer muchas
básicos del cuerpo en el que vive. veces al día o que es importante esti­
rar antes de entrenar) han sido des­
terradas por la ciencia más reciente,
pero siguen formando parte de las
recomendaciones habituales.
03 04
Recomendaciones prácticas para Una visi

In [23]:
# 2️⃣ CREACIÓN DE EMBEDDINGS Y FAISS
# ------------------------------------

# Cargar el modelo preentrenado de SentenceTransformers
embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Crear los embeddings a partir de los documentos
document_embeddings = embedder.encode(documents, convert_to_numpy=True)

# Crear un índice FAISS
index = faiss.IndexHNSWFlat(document_embeddings.shape[1], 32)
index.add(document_embeddings)

# Guardar los embeddings y el índice FAISS
with open("document_embeddings.pkl", "wb") as f:
    pickle.dump(document_embeddings, f)
faiss.write_index(index, "faiss_index.faiss")

# Log para confirmar que se guardaron correctamente
logging.info("Embeddings e índice FAISS guardados correctamente.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
# 3️⃣ FUNCIONES PARA RAG
# ------------------------------------

def retrieve_documents(query, top_k=3):
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    return [documents[i] for i in indices[0] if i < len(documents)]

def answer_question_with_generation(question, context):
    try:
        inputs = tokenizer(question, context, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            start, end = outputs.start_logits.argmax(), outputs.end_logits.argmax()
            return tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0][start:end+1]))
    except Exception as e:
        logging.error(f"Error en la generación de respuesta: {e}")
        return "No se pudo generar la respuesta."

def answer_question_with_RAG(question, top_k=3):
    try:
        context = " ".join(retrieve_documents(question, top_k))
        return answer_question_with_generation(question, context)
    except Exception as e:
        logging.error(f"Error en el proceso RAG: {e}")
        return "Error en la generación de respuesta."

In [ ]:
# 4️⃣ GUARDAR Y SUBIR MODELO A HUGGING FACE
# ------------------------------------

rag_model_dir = "/content/chatbot_tribu_salud_rag"
os.makedirs(rag_model_dir, exist_ok=True)

try:
    model.save_pretrained(rag_model_dir)
    tokenizer.save_pretrained(rag_model_dir)
    logging.info(f"Modelo RAG guardado en: {rag_model_dir}")
except Exception as e:
    logging.error(f"Error al guardar el modelo: {e}")

repo_name_rag = "CasiAC/chatbot_tribu_salud_rag"
repo_dir_rag = "/content/new_chatbot_tribu_salud_rag"
zip_filename = "/content/chatbot_tribu_salud_rag.zip"

if not os.path.exists(rag_model_dir):
    logging.error(f"No se encontró la carpeta {rag_model_dir}")
    raise FileNotFoundError(f"No se encontró la carpeta {rag_model_dir}")

api = HfApi()

try:
    api.create_repo(repo_name_rag, repo_type="model", private=False, exist_ok=True)
    if os.path.exists(repo_dir_rag):
        shutil.rmtree(repo_dir_rag)
    os.makedirs(repo_dir_rag, exist_ok=True)
    shutil.copytree(rag_model_dir, repo_dir_rag, dirs_exist_ok=True)
    api.upload_folder(folder_path=repo_dir_rag, repo_id=repo_name_rag, repo_type="model", commit_message="Subida del modelo RAG")
    logging.info(f"Modelo RAG subido correctamente a Hugging Face: https://huggingface.co/{repo_name_rag}")
except Exception as e:
    logging.error(f"Error al subir el modelo a Hugging Face: {e}")

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, filenames in os.walk(rag_model_dir):
        for file in filenames:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, rag_model_dir)
            zipf.write(file_path, arcname)

print(f"✅ Modelo RAG guardado y comprimido en: {zip_filename}")
files.download(zip_filename)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

✅ Modelo RAG guardado y comprimido en: /content/chatbot_tribu_salud_rag.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Nombre del modelo en Hugging Face
MODEL_NAME = "CasiAC/chatbot_tribu_salud_rag"

# Cargar el modelo y el tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

# Crear el pipeline para generación de texto
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)


tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

Device set to use cuda:0


In [28]:
from huggingface_hub import model_info

model_name = "CasiAC/chatbot_tribu_salud_rag"

try:
    info = model_info(model_name)
    print(f"✅ Modelo encontrado: {info.modelId}")
except Exception as e:
    print(f"❌ Error: {e}")


✅ Modelo encontrado: CasiAC/chatbot_tribu_salud_rag


In [29]:
# Configurar logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Documentos de ejemplo
documents = [
    "Este documento contiene información clave sobre salud evolutiva.",
    "Consejos prácticos sobre entrenamiento funcional y movimiento natural.",
    "Principios fundamentales de nutrición y estilo de vida saludable.",
    "La importancia del descanso y la regulación del estrés en la salud ancestral.",
    "El impacto de la alimentación basada en alimentos no procesados y dieta cetogénica.",
    "Rutinas efectivas con kettlebells para ganar fuerza y resistencia."
]

# Inicializar modelo de embeddings
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Convertir documentos en embeddings y construir índice FAISS
document_embeddings = embedding_model.encode(documents, normalize_embeddings=True)
index = faiss.IndexFlatL2(document_embeddings.shape[1])
index.add(np.array(document_embeddings))

# Mensaje de bienvenida optimizado
system_message = "¡Bienvenido a Tribu Salud! Soy tu asistente de salud evolutiva. Pregunta sobre alimentación, entrenamiento y descanso. ¡Estoy aquí para ayudarte!"

# Nombre del modelo en Hugging Face
MODEL_NAME = "CasiAC/chatbot_tribu_salud_rag"

# Forzar uso de CPU (puedes cambiar a GPU si es necesario)
device = torch.device("cpu")
logging.info("Usando CPU para la inferencia.")

# Cargar modelo y tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="cpu")
tokenizer.pad_token = tokenizer.eos_token

# Pipeline de generación de texto
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, batch_size=1)

# Función para generar respuesta con formato limpio
def generate_response(prompt, max_new_tokens=400):
    full_prompt = f"<s>[INST] {system_message}\n{prompt}\nPor favor, responde en español. [/INST]"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.6,  # Ajustado para mayor coherencia
            top_p=0.85        # Reduce la aleatoriedad excesiva
        )

    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    response = response.replace(full_prompt, "").strip()

    # Limpiar salida y eliminar partes redundantes
    response = response.split("[INST]")[-1].strip()
    response = response.split("[/INST]")[-1].strip()

    return response

# Función para generar respuesta con contexto filtrado
def answer_question_with_generation(question, context, max_context_tokens=200):
    # Limitar tokens del contexto para evitar respuestas demasiado largas o irrelevantes
    context_tokens = tokenizer.encode(context, add_special_tokens=False)
    if len(context_tokens) > max_context_tokens:
        context_tokens = context_tokens[:max_context_tokens]
        context = tokenizer.decode(context_tokens, skip_special_tokens=True)

    prompt = f"Pregunta: {question}\nContexto relevante: {context}"
    return generate_response(prompt)

# Función para recuperar documentos relevantes con embeddings
def retrieve_documents(question, top_k=2):
    question_embedding = embedding_model.encode([question], normalize_embeddings=True)
    distances, indices = index.search(np.array(question_embedding), top_k)
    retrieved_docs = [documents[i] for i in indices[0] if i < len(documents)]
    return retrieved_docs

# Función para prueba del modelo RAG con contexto filtrado
def test_rag_model(question, top_k=2):
    try:
        context = " ".join(retrieve_documents(question, top_k))
        answer = answer_question_with_generation(question, context)
        logging.info(f"Pregunta: {question}\nContexto: {context}\nRespuesta: {answer}")
        return answer
    except Exception as e:
        logging.error(f"Error en la prueba del modelo RAG: {e}")
        return "❌ Ocurrió un error en la prueba del modelo."

# Interfaz Gradio optimizada
def chatbot_interface(question):
    return test_rag_model(question)

gr.Interface(
    fn=chatbot_interface,
    inputs="text",
    outputs="text",
    title="Tribu Salud 🦴🌿 - Chatbot de Salud Evolutiva",
    description="Pregunta sobre salud evolutiva, entrenamiento funcional, nutrición y estilo de vida saludable.",
).launch(share=True)


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1b3dad2652423da5e2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
